# 運動方程式の線形化の数式処理

吉田勝俊（宇都宮大学）

### 参考情報
- [Welcome to SymPy’s documentation! &#8212; SymPy 1.10.1 documentation](https://docs.sympy.org/) （本家）
- [SymPy による数式処理とグラフ作成 - 弘前大学 Home Sweet Home](https://home.hirosaki-u.ac.jp/jupyter/sympy/)

In [ ]:
import sympy as sym #数式処理ライブラリ

## ◯オイラー・ラグランジュ方程式

### 定義：

$\displaystyle
\frac{d}{dt}\left(\frac{\partial L}{\partial \dot q_i}\right)
-\frac{\partial L}{\partial q_i}
+\frac{\partial D}{\partial \dot q_i}
-Q_i
= 0, \quad i=1,2,\cdots
$

In [ ]:
def Derive_ELE(system):
    '''
    オイラー・ラグランジュ方程式（ELE: Euler-Lagrange equation）
    「左辺＝０」の左辺を導出する関数
    
    def system():    
        t: 時間（スカラ）
        q: 一般化座標（ベクトル）
        L: ラグランジュ関数（スカラ）
        D: 散逸関数（スカラ）
        Q: 一般化力（ベクトル）
        u: 外部入力（ベクトル）※線形化で用いる
        plist: パラメータ（リスト）
        
        return {'t':t, 'q':q, 'L':L, 'D':D, 'Q':Q, 'u':u, 'plist':plist} #辞書型
    '''
    t, q, L, D, Q, u = [ #plistは未使用
        system()[key]
        for key in ['t','q','L','D','Q','u']
    ]
    dqdt             = sym.diff(q, t) #一般化座標の時間微分
    dim              = len(q)         #一般化座標の次元
    
    ### ラグランジュの運動方程式 ###
    EOM = sym.Matrix([
        sym.simplify(
            sym.diff( sym.diff(L, dqdt[i]), t )
            - sym.diff(L, q[i])
            + sym.diff(D, dqdt[i])
            - (Q[i] if Q is not None else 0) #Q が無ければ 0
        )
        for i in range(dim)
    ])
    
    ELE = {'EOM':EOM}
    ELE.update(system()) #system()の結果を継承

    return ELE

## ◯運動方程式の線形化と１階化

### ■ユーザー関数

In [ ]:
def initial_state(x0, x0symb, x):
    '''
    線形化の基準点が未指定なら記号で生成する
    '''
    if x0 is None:
        if len(x) == 1:  #基準点がスカラ（1次元）の場合
            x0 = sym.Matrix([x0symb])
        else:            #基準点がベクトル（多次元）の場合
            x0 = sym.Matrix([
                sym.symbols(x0symb + '_' + str(i+1)) for i in range(len(x))
            ])

    else:
        x0 = sym.Matrix(x0)

    return x0

In [ ]:
def Linearized_ELE(ELE, q0=None, u0=None):
    '''
    運動方程式を，x0 の近傍で線形化する
    - ELE := derive_ELE(system)の出力
    '''
    EOMq, q, t, u = [
        ELE[key]
        for key in ['EOM','q','t','u']
    ]

    #
    # 一般化座標の微分
    #
    dq  = sym.Matrix([sym.diff(qi, t) for qi in q])    #1階微分
    ddq = sym.Matrix([sym.diff(qi, t, 2) for qi in q]) #2階微分
    
    #
    # ヤコビ行列
    #
    M = EOMq.jacobian(ddq) #慣性行列：加速度に関するヤコビ行列
    C = EOMq.jacobian(dq)  #減衰行列：速度に関するヤコビ行列
    K = EOMq.jacobian(q)   #剛性行列：変位に関するヤコビ行列
    
    #入力行列 ※存在すれば計算．これだけ右辺なのでマイナス　
    U = -EOMq.jacobian(u) if u is not None else None

    # 平衡点の準備
    q0 = initial_state(q0, 'q0', q)
    u0 = initial_state(u0, 'u0', u) if u is not None else None

    # 平衡点と平衡条件の代入
    def subs_equilibrium(mat):
        '''
        平衡点と平衡条件を代入する
        '''
        for i in range(len(q)):
            mat = mat.subs(ddq[i], 0)
            mat = mat.subs(dq[i], 0)
            mat = mat.subs(q[i], q0[i])
        
        if u is not None:
            for i in range(len(u)):
                mat = mat.subs(u[i], u0[i]) 
            
        return sym.simplify(mat)
    
    M = subs_equilibrium(M)
    C = subs_equilibrium(C)
    K = subs_equilibrium(K)
    if u is not None:
        U = subs_equilibrium(U)
    
    ### 結果 ###
    LELE = {'M':M, 'C':C, 'K':K, 'U':U, 'q0':q0, 'u0':u0}
    LELE.update(ELE) #継承
    
    return LELE

In [ ]:
def FirstOrdered_LELE(LELE):
    '''
    線形化した運動方程式を１階化する
    - LELE ... Linearized_ELE(ELE, q0=None, u0=None)の出力
    '''
    q = LELE['q']
    u = LELE['u']
    t = LELE['t']

    ### 状態ベクトルの生成 ###
    # 実体
    xq = sym.Matrix(
        [qi for qi in q] 
        + #リストの + は連結
        [sym.diff(qi, t) for qi in q]
    )

    # 通し番号
    x = sym.Matrix([
        sym.Function(r'x_' + str(i+1), real=True)(t)
        for i in range(len(xq))
    ])

    ### 状態ベクトルに関するヤコビ行列 ###
    qdim = len(q)
    E    = sym.eye(qdim)        #qdim×qdim 単位行列
    Oq   = sym.zeros(qdim,qdim) #qdim×qdim ゼロ行列
    ovec = sym.zeros(qdim,1)    #qdimゼロベクトル
    invM = sym.Inverse(LELE['M'])  #質量行列の逆行列
    
    # 平衡点
    dfxdx = sym.Matrix([
        [Oq,              E              ],
        [-invM*LELE['K'], -invM*LELE['C']]
    ])
    dfxdx = sym.simplify(dfxdx)
    
    # 平衡点
    x0 = sym.Matrix([
        [LELE['q0']],
        [ovec]
    ])

    # 平衡点
    jacobi_x = {'A':dfxdx, 'x0':x0} #(ヤコビ行列，基準点)

    ### 外部入力に関するヤコビ行列 ###
    if u is not None: #もし外部入力が存在すれば
        udim = len(u)
        Ou = sym.zeros(qdim,udim) #qdim×udim ゼロ行列
        
        dfxdu = sym.Matrix([
            [Ou],
            [invM*LELE['U']]
        ])
        dfxdu = sym.simplify(dfxdu)
        
        jacobi_u = {'B':dfxdu, 'u0':LELE['u0']} #(ヤコビ行列，基準点)
        
    else:
        jacobi_u = {'B':None, 'u0':None}

    
    ### 結果（辞書型） ###
    linearized = {'x':x, 'xq':xq}  #状態ベクトルの定義
    linearized.update(jacobi_u)    #xの(ヤコビ行列，基準点)をマージ
    linearized.update(jacobi_x)    #uの(ヤコビ行列，基準点)をマージ
    linearized.update(LELE)        #継承
        
    return linearized

### ■単振り子（SP: simple pendulum）

- 一般化座標: $(\theta(t))$

In [ ]:
def system_SP():
    '''
    システムを定義する関数（単振り子）
    '''
    # パラメータ: 
    t, m, l, g = sym.symbols(
        't m l g', 
        positive=True #正の実数に制限 ※なるべく制限すると simplify がよく効く．以下同．
    )
    c = sym.symbols('c', positive=True)
    plist = [m, l, g, c] #パラメータのリスト
    
    # 一般化座標（時間関数）: 
    q = sym.Matrix([
        sym.Function(
            r'\theta', 
            real=True #実数値に制限
        )(t),
    ])
    
    # 質点の直交座標
    theta = q[0]
    xx = l*sym.Matrix([
        sym.sin(theta),
        -sym.cos(theta),
    ])
    
    dxxdt = sym.diff(xx, t) #その時間微分
    
    # 運動エネルギー
    T = (m/2)*dxxdt.dot(dxxdt)
    
    # 位置エネルギー
    h = xx[1] #振子先端の高さ
    U = m*g*h
    
    # ラグランジュ関数
    L = T - U

    # 散逸関数
#     D = (c/2)*dxxdt.dot(dxxdt) #粘性減衰
    D = 0 #減衰なし

    # 外部入力（この系は無し）
    u = None

    # 一般化力
    Q = u
    
    return {'t':t, 'q':q, 'L':L, 'D':D, 'Q':Q, 'u':u, 'plist':plist}

#### 運動方程式の導出結果

In [ ]:
ele_SP = Derive_ELE(system_SP)

display(ele_SP['EOM'])

#### 線形化の結果

In [ ]:
lele_SP = Linearized_ELE(ele_SP, q0=[0])

for _ in ['M', 'C', 'K', 'U']:
    display(lele_SP[_])

上の出力より，平衡点 $q^\ast=0$の近傍で線形化した運動方程式は次のようになります．

$$
ml^2 \ddot\varphi + mgl\varphi = 0
$$

#### 線形化した運動方程式の１階化

In [ ]:
first_SP = FirstOrdered_LELE(lele_SP)

for _ in ['xq', 'A', 'x0', 'B', 'u0']:
    display(first_SP[_])

先に１階化してから線形化したのと同じ結果が得られています．

### ■台車型倒立振子（CIP: cart inverted pendulum）

- 一般化座標: $(x(t),\theta(t))$

In [ ]:
def system_CIP():
    '''
    システムを定義する関数 （問題に応じて書き換える）
    ※以下は，台車型倒立振子の例
    '''
    # パラメータ: 
    t, M, m, l, g, c1, c2 = sym.symbols(
        't M m l g c_1 c_2', 
        positive=True #正の実数に制限 ※なるべく制限すると simplify がよく効く．以下同．
    )
    G, S = sym.symbols('G S', positive=True)   #消去されるパラメータ
    plist = [M, m, l, g, c1, c2] #パラメータのリスト
    
    # 一般化座標（時間関数）: 
    q = sym.Matrix([
        sym.Function(
            r'x', 
            real=True #実数値に制限
        )(t),
        sym.Function(
            r'\theta', 
            real=True #実数値に制限
        )(t),
    ])
    
    # 質点の直交座標
    x, th = q
    xM = sym.Matrix([
        x,
        G
    ])
    xm = sym.Matrix([
        x + l*sym.sin(th),
        l*sym.cos(th) + S,
    ])
    
    dxMdt = sym.diff(xM, t) #その時間微分
    dxmdt = sym.diff(xm, t)
    
    # 運動エネルギー
    T = (M/2)*dxMdt.dot(dxMdt) + (m/2)*dxmdt.dot(dxmdt)
    
    # 位置エネルギー
    h = xm[1] #振子先端の高さ
    U = m*g*h
    
    # ラグランジュ関数
    L = T - U

    # 散逸関数
    D = (c1/2)*sym.diff(x,t)**2 + (c2/2)*sym.diff(th,t)**2 #粘性減衰
#     D = 0 #減衰なし

    # 外部入力（この系は１個だけ）
    u = sym.Matrix([
        sym.Function(r'u', real=True)(t)
    ])
     
    # 一般化力（外部入力が第１成分のみに作用）
    Q = sym.Matrix([
        u[0],
        0
    ])
    
    return {'t':t, 'q':q, 'L':L, 'D':D, 'Q':Q, 'u':u, 'plist':plist}

#### 運動方程式の導出結果

In [ ]:
ele_CIP = Derive_ELE(system_CIP)

display(ele_CIP['EOM']) #運動方程式（左辺＝０）の左辺

#### 線形化の結果

In [ ]:
lele_CIP = Linearized_ELE(ele_CIP) #平衡点未指定

for _ in ['M', 'C', 'K', 'U']:
    display(lele_CIP[_])

In [ ]:
lele_CIP0 = Linearized_ELE(ele_CIP, [0,0], [0]) #平衡点指定

for _ in ['M', 'C', 'K', 'U']:
    display(lele_CIP0[_])

上の出力より，平衡点 $\boldsymbol{q}^\ast=(0,0)$の近傍で線形化した運動方程式は次のようになります．

$$
\def\bmat#1{\begin{bmatrix}#1\end{bmatrix}}
    \bmat{M+m & ml\\ ml & ml^2} \bmat{\ddot\varphi_1 \\ \ddot\varphi_2}
   +\bmat{c_1 & 0 \\ 0 & c_2}   \bmat{\dot\varphi_1 \\ \dot\varphi_2}
   +\bmat{0 & 0 \\ 0 & -mgl}    \bmat{\varphi_1 \\ \varphi_2}
  =
    \bmat{1 \\ 0} \mu(t)
$$

#### 線形化した運動方程式の１階化

In [ ]:
first_CIP = FirstOrdered_LELE(lele_CIP) #平衡点未指定

for _ in ['xq', 'A', 'x0', 'B', 'u0']:
    display(first_CIP[_])

In [ ]:
first_CIP0 = FirstOrdered_LELE(lele_CIP0) #平衡点指定

for _ in ['xq', 'A', 'x0', 'B', 'u0']:
    display(first_CIP0[_])

先に１階化してから線形化したのと同じ結果が得られています．

## ◯平衡点の導出《おまけ》

In [ ]:
def Equilibrium(ELE, u0=None):
    '''
    運動方程式を，x0 の近傍で線形化する
    - ELE := derive_ELE(system)の出力
    '''
    EOMq, q, t, u = [
        ELE[key]
        for key in ['EOM','q','t','u']
    ]

    #
    # 一般化座標の微分
    #
    dq  = sym.Matrix([sym.diff(qi, t) for qi in q])    #1階微分
    ddq = sym.Matrix([sym.diff(qi, t, 2) for qi in q]) #2階微分
    
    display(EOMq)
    #
    # ヤコビ行列
    #
    M = EOMq.jacobian(ddq) #慣性行列：加速度に関するヤコビ行列
    
    #
    # 右辺の関数 F
    #
    F = sym.simplify(EOMq - M*ddq)
    
    #
    # 平衡条件の代入
    #
    q0 = initial_state(None, 'q0', q)
    u0 = initial_state(u0, 'u0', u) if u is not None else None

    for i in range(len(q)):
        F = F.subs(dq[i], 0)
        F = F.subs(q[i], q0[i])

    if u is not None:
        for i in range(len(u)):
            F = F.subs(u[i], u0[i])

    F = sym.simplify(F)

    #
    # 平衡点を求める
    #
    sol = sym.solve(F, q0, dict=True)
    
    return (F, q0, sol)

#### 台車型倒立振子の平衡点

In [ ]:
F_CIP, q0_CIP, sol_CIP = Equilibrium(ele_CIP)
display(F_CIP, q0_CIP, sol_CIP)

### 得られた平衡点を若干見やすく出力

In [ ]:
for sol in sol_CIP:
    for q in q0_CIP:
        try:
            s = sol[q]
            display(q,s)
            print('---')
        except:
            pass